In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/processed-usa-less/processed_USA_less.csv


In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.read_csv("/kaggle/input/processed-usa-less/processed_USA_less.csv")
df.head()

,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),City,State,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,datetime_start_parsed,datetime_end_parsed,datetime_weathertime_parsed
0,3,39.865147,-84.058723,39.865147,-84.058723,0.01,Dayton,OH,62.0,62.0,91,29.68,10.0,7.0,0.02,Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,2016-02-08 05:46:00,2016-02-08 11:00:00,2016-02-08 05:58:00
1,2,39.928059,-82.831184,39.928059,-82.831184,0.01,Reynoldsburg,OH,62.0,62.0,100,29.65,10.0,7.0,0.00,Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,2016-02-08 06:07:59,2016-02-08 06:37:59,2016-02-08 05:51:00
2,2,39.063148,-84.032608,39.063148,-84.032608,0.01,Williamsburg,OH,36.0,33.3,100,29.67,10.0,3.5,0.00,Cloudy Skies,False,False,False,False,False,False,False,False,False,False,False,True,False,2016-02-08 06:49:27,2016-02-08 07:19:27,2016-02-08 06:56:00
3,3,39.747753,-84.205582,39.747753,-84.205582,0.01,Dayton,OH,35.1,31.0,96,29.64,9.0,4.6,0.00,Cloudy Skies,False,False,False,False,False,False,False,False,False,False,False,False,False,2016-02-08 07:23:34,2016-02-08 07:53:34,2016-02-08 07:38:00
4,2,39.627781,-84.188354,39.627781,-84.188354,0.01,Dayton,OH,36.0,33.3,89,29.65,6.0,3.5,0.00,Cloudy Skies,False,False,False,False,False,False,False,False,False,False,False,True,False,2016-02-08 07:39:07,2016-02-08 08:09:07,2016-02-08 07:53:00


In [5]:
# List of columns to convert to datetime
columns_to_convert = ['datetime_start_parsed','datetime_end_parsed', 'datetime_weathertime_parsed']

# Convert specified columns to datetime
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_datetime, errors='coerce')


In [6]:
df=df.convert_dtypes()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7700098 entries, 0 to 7700097
Data columns (total 32 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   Severity                     Int64         
 1   Start_Lat                    Float64       
 2   Start_Lng                    Float64       
 3   End_Lat                      Float64       
 4   End_Lng                      Float64       
 5   Distance(mi)                 Float64       
 6   City                         string        
 7   State                        string        
 8   Temperature(F)               Float64       
 9   Wind_Chill(F)                Float64       
 10  Humidity(%)                  Int64         
 11  Pressure(in)                 Float64       
 12  Visibility(mi)               Float64       
 13  Wind_Speed(mph)              Float64       
 14  Precipitation(in)            Float64       
 15  Weather_Condition            string        
 16  

In [7]:
columns_to_drop = ["City"]
df = df.drop(columns=columns_to_drop, errors='ignore')

In [8]:
severity_mapping = {
    1: 0,
    2: 0,
    3: 1,
    4: 1
}

df['Severity'] = df['Severity'].map(severity_mapping)

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Randomly shuffle the rows of the DataFrame
df_shuffled = df.sample(frac=1, random_state=42)

# Split the shuffled dataset into 5 equal parts
df_parts = [df_shuffled.iloc[i:i + len(df_shuffled)//5] for i in range(0, len(df_shuffled), len(df_shuffled)//5)]

# Choose one part for analysis (for example, the first part)
df_selected = df_parts[0]

# Step 1: Handling Categorical Data
df_encoded = pd.get_dummies(df_selected, columns=['Weather_Condition', 'State'])

# Step 2: Scaling Latitude and Longitude
scaler = MinMaxScaler()
df_encoded[['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng']] = scaler.fit_transform(df_encoded[['Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng']])

# Extract features from datetime columns
df_encoded['start_hour'] = df_encoded['datetime_start_parsed'].dt.hour
df_encoded['start_dayofweek'] = df_encoded['datetime_start_parsed'].dt.dayofweek
df_encoded['start_month'] = df_encoded['datetime_start_parsed'].dt.month

df_encoded['end_hour'] = df_encoded['datetime_end_parsed'].dt.hour
df_encoded['end_dayofweek'] = df_encoded['datetime_end_parsed'].dt.dayofweek
df_encoded['end_month'] = df_encoded['datetime_end_parsed'].dt.month

df_encoded['weathertime_hour'] = df_encoded['datetime_weathertime_parsed'].dt.hour
df_encoded['weathertime_dayofweek'] = df_encoded['datetime_weathertime_parsed'].dt.dayofweek
df_encoded['weathertime_month'] = df_encoded['datetime_weathertime_parsed'].dt.month

# Drop original datetime columns
df_encoded = df_encoded.drop(columns=['datetime_start_parsed', 'datetime_end_parsed', 'datetime_weathertime_parsed'])

# Step 3: Splitting Data and Training a Simple Model
# Assuming 'Severity' is the target variable
X = df_encoded.drop('Severity', axis=1)
y = df_encoded['Severity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy.stats import randint

# Define the parameter distribution to sample from
param_dist = {
    'n_estimators': 200,
    'max_depth': 20,
}

# Create a RandomForestClassifier
model = RandomForestClassifier(random_state=42)

# Create RandomizedSearchCV
random_search = RandomizedSearchCV(
    model, param_distributions=param_dist, n_iter=20, cv=5, scoring='accuracy', random_state=42
)

# Fit the model to the training data
random_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", random_search.best_params_)

# Get the best model
best_model_random = random_search.best_estimator_

# Predictions on the test set using the best model
y_pred_random_tuned = best_model_random.predict(X_test)

# Evaluate the tuned model
accuracy_random_tuned = accuracy_score(y_test, y_pred_random_tuned)
print(f"Tuned Model Accuracy: {accuracy_random_tuned:.2f}")

# Additional evaluation metrics for the tuned model
print("Classification Report (Tuned Model):")
print(classification_report(y_test, y_pred_random_tuned))

# Display confusion matrix
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test, y_pred_random_tuned))


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Set the desired hyperparameter values
param_dist = {
    'n_estimators': 200,
    'max_depth': 20,
}

# Create a RandomForestClassifier with fixed hyperparameters
model = RandomForestClassifier(random_state=42, **param_dist)

# Create RandomizedSearchCV with n_iter set to 1
random_search = RandomizedSearchCV(
    model, param_distributions={}, n_iter=1, cv=5, scoring='accuracy', random_state=42
)

# Fit the model to the training data
random_search.fit(X_train, y_train)

# Print the best hyperparameters (should match the fixed values)
print("Best Hyperparameters:", random_search.best_params_)

# Get the best model
best_model_random = random_search.best_estimator_

# Predictions on the test set using the best model
y_pred_random_tuned = best_model_random.predict(X_test)

# Evaluate the tuned model
accuracy_random_tuned = accuracy_score(y_test, y_pred_random_tuned)
print(f"Tuned Model Accuracy: {accuracy_random_tuned:.2f}")

# Additional evaluation metrics for the tuned model
print("Classification Report (Tuned Model):")
print(classification_report(y_test, y_pred_random_tuned))

# Display confusion matrix
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test, y_pred_random_tuned))


Best Hyperparameters: {}
Tuned Model Accuracy: 0.85
Classification Report (Tuned Model):
              precision    recall  f1-score   support

           0       0.85      0.98      0.91    248531
           1       0.79      0.29      0.43     59473

    accuracy                           0.85    308004
   macro avg       0.82      0.64      0.67    308004
weighted avg       0.84      0.85      0.82    308004

Confusion Matrix for Random Forest Classifier:
[[243902   4629]
 [ 42012  17461]]
